# 03 –  Ricavare dati da infrastrutture spaziali

## Introduzione

Questo notebook illustra come accedere e manipolare dati geospaziali da diverse **infrastrutture pubbliche**. Imparerete a:

- Interrogare servizi web standardizzati (OGC, WFS, ArcGIS REST)
- Estrarre dati da OpenStreetMap tramite Overpass API
- Processare e visualizzare dati geografici
- Applicare analisi spaziali pratiche

### Servizi coperti:
1. **OGC API Features** - Standard moderno per API geospaziali
2. **WFS (Web Feature Service)** - Standard OGC per dati vettoriali
3. **ArcGIS REST** - API proprietaria Esri molto diffusa
4. **Overpass/OSM** - Interrogazione di OpenStreetMap
5. **GeoJSON via HTTP** - Formato JSON per dati geografici



In [20]:
# Installazione (se serve)
#%pip install -q geopandas pyogrio requests OWSLib folium pyproj shapely overpass

In [ ]:
#in JupyterLite

import micropip
# librerie pure-python che funzionano in Pyodide
await micropip.install([
    "folium",
    "OWSLib",
    "overpy",          # client Overpass (HTTP)
    "pyshp",           # per leggere shapefile
    "pyodide-http>=0.2.1"  # patch rete per requests/urllib
])

import pyodide_http
pyodide_http.patch_all()  # abilita requests/urllib via fetch (soggetto a CORS)


## Funzioni Helper - Utilità riutilizzabili

Creiamo alcune funzioni di utilità per semplificare le operazioni ripetitive:

### `fetch_json()`
Scarica JSON da un URL con gestione errori e retry automatici. Utile perché i servizi web possono essere instabili.

### `gdf_preview()`
Mostra un'anteprima strutturata di un GeoDataFrame: CRS, numero righe, colonne, primi record.

### `safe_save()`
Salva dati in due formati:
- **GeoPackage (.gpkg)**: Standard OGC, supporta geometrie complesse
- **Parquet (.parquet)**: Formato colonnare compresso, ottimo per analisi

In [3]:
# Import + helper functions
import os, io, json, time, urllib.parse as up
from pathlib import Path
import requests, pandas as pd, geopandas as gpd
from shapely.geometry import Point, box
from IPython.display import display
import folium

# Directory per salvare i risultati
OUTDIR = Path("data/data_lezione_03")
OUTDIR.mkdir(exist_ok=True)

# Header HTTP per identificarsi correttamente ai server
DEFAULT_HEADERS = {"User-Agent": "LezioneSpaziale/1.0 (+https://example.org)"}
print(f"[OK] Output dir: {OUTDIR.resolve()}")

def fetch_json(url, params=None, retries=2, timeout=40):
    """Scarica JSON con retry automatico in caso di errori temporanei"""
    last = None
    for i in range(1, retries+1):
        try:
            print(f"[fetch_json] {i}/{retries} → {url}")
            r = requests.get(url, params=params, headers=DEFAULT_HEADERS, timeout=timeout)
            r.raise_for_status()
            print("[fetch_json] OK")
            return r.json()
        except Exception as e:
            print("[fetch_json] Errore:", e)
            last = e
            time.sleep(min(2*i, 8))  # Backoff esponenziale
    raise last

def gdf_preview(gdf, n=5, name="(df)"):
    """Mostra informazioni strutturate su un GeoDataFrame"""
    print(f"[preview] {name} | CRS:", gdf.crs, "| righe:", len(gdf))
    print("Colonne:", list(gdf.columns))
    if len(gdf):
        display(gdf.head(n))
    if hasattr(gdf, "total_bounds"):
        print("Bounds:", gdf.total_bounds)

def safe_save(gdf, base):
    """Salva in GeoPackage (standard) e Parquet (performance)"""
    gpkg = OUTDIR/f"{base}.gpkg"
    pq = OUTDIR/f"{base}.parquet"
    gdf.to_file(gpkg, driver="GPKG")
    gdf.to_parquet(pq, index=False)
    print("[save] ", gpkg, "e", pq)

[OK] Output dir: /home/ciro/Downloads/Corso_PyQGIS/data/data_lezione_03


---

## 1) OGC API Features (pygeoapi demo)

### Cos'è OGC API Features?

È lo **standard moderno** dell'Open Geospatial Consortium per accedere a dati geografici vettoriali via web. Sostituisce il vecchio WFS con un approccio RESTful più semplice.

### Caratteristiche principali:
- API RESTful (usa HTTP GET)
- Output in **GeoJSON** (formato JSON standard per geometrie)
- Navigazione tramite link (HATEOAS)
- Paginazione automatica

### Struttura endpoint:
```
/collections              → Lista collezioni disponibili
/collections/{id}/items   → Features della collezione
```

### Cosa facciamo:
1. Leggiamo l'elenco delle collezioni disponibili
2. Selezioniamo una collezione
3. Scarichiamo i dati in formato GeoJSON
4. Convertiamo in GeoDataFrame per analisi Python

In [4]:
# URL base del servizio demo di pygeoapi
OAPIF = "https://demo.pygeoapi.io/master"

# Leggi elenco collezioni
cols = fetch_json(f"{OAPIF}/collections").get("collections", [])
print("[INFO] collections:", len(cols))

# Mostra i primi 10 ID
[c.get("id") for c in cols[:10]]

[fetch_json] 1/2 → https://demo.pygeoapi.io/master/collections
[fetch_json] OK
[INFO] collections: 17


['obs',
 'lakes',
 'dutch_windmills',
 'dutch_castles',
 'dutch_georef_stations',
 'utah_city_locations',
 'unesco_pois_italy',
 'ogr_gpkg_poi',
 'ogr_geojson_lakes',
 'ogr_gpkg_wales_railway_lines']

### Selezione e download collezione

Scegliamo una collezione e scarichiamo le sue features. Il parametro `limit=200` richiede fino a 200 elementi (alcuni server hanno limiti più bassi).

In [ ]:
# Selezione automatica di una collezione Natural Earth
target = "unesco_pois_italy" 
for c in cols:
    cid = c.get("id") or c.get("name")
    if cid and cid.startswith("ne_"):
        target = cid
        break
if not target and cols:
    target = cols[0].get("id") or cols[0].get("name")
    
print("[SCELTA] collection:", target)

# 2. Scarica items della collezione
items = fetch_json(f"{OAPIF}/collections/{target}/items", params={"limit": 200})

# 3. Converti da GeoJSON a GeoDataFrame
feats = items.get("features", [])
gdf_oapi = gpd.GeoDataFrame.from_features(
    {"type": "FeatureCollection", "features": feats}, 
    crs="EPSG:4326"  # Sistema di coordinate geografiche (lat/lon)
)

gdf_preview(gdf_oapi, name=f"oapi:{target}")
#safe_save(gdf_oapi, f"oapi_{target}")

[SCELTA] collection: unesco_pois_italy
[fetch_json] 1/2 → https://demo.pygeoapi.io/master/collections/unesco_pois_italy/items
[fetch_json] OK
[preview] oapi:unesco_pois_italy | CRS: EPSG:4326 | righe: 10
Colonne: ['geometry', 'cod_unesco', 'sito', 'seriale', 'cod_compon', 'componente', 'tipo_area']


,geometry,cod_unesco,sito,seriale,cod_compon,componente,tipo_area
0,POINT (12.6382 43.095),IT_549 rev,Il Palazzo Reale del XVIII secolo di Caserta c...,0,None,Acquedotto,sito
1,POINT (7.70847 45.05813),IT_823,Residenze Sabaude,1,823-005,Villa della Regina,sito
2,POINT (12.5049 41.8852),IT/VA_91bis,"Centro storico di Roma, le proprietà extraterr...",1,091-009(b),Palazzi detti dei Propilei (sud),sito
3,POINT (12.5074 41.8873),IT/VA_91bis,"Centro storico di Roma, le proprietà extraterr...",1,091-009(a),Palazzi detti dei Propilei (nord),sito
4,POINT (12.4892 41.893),IT/VA_91bis,"Centro storico di Roma, le proprietà extraterr...",1,091-010,Palazzo Pio,sito


Bounds: [ 7.70847062 41.8852     12.6382     45.05813397]
[save]  data/data_lezione_03/oapi_unesco_pois_italy.gpkg e data/data_lezione_03/oapi_unesco_pois_italy.parquet


---

## 2) WFS (PCN, GML) – Bacini idrografici principali

### Cos'è WFS (Web Feature Service)?

È uno **standard OGC più vecchio** (ma ancora molto usato) per servire dati vettoriali. A differenza di OGC API Features, usa protocolli più complessi.

### Caratteristiche:
- Output in **GML** (Geography Markup Language, formato XML)
- Supporta query complesse (filtri CQL/OGC Filter)
- Versioni: 1.0.0, 1.1.0, 2.0.0
- Richiede libreria client (OWSLib)

### PCN (Portale Cartografico Nazionale)
Il PCN del Ministero dell'Ambiente italiano espone molti dataset tramite WFS, inclusi i bacini idrografici.

### Cosa facciamo:
1. Connettiamo al servizio WFS con OWSLib
2. Richiediamo il layer dei bacini idrografici principali
3. GeoPandas legge automaticamente il GML e lo converte

In [1]:
# from owslib.wfs import WebFeatureService

# # URL del servizio WFS del PCN
# BASE_PCN = "https://wms.pcn.minambiente.it/ogc?map=/ms_ogc/wfs/Bacini_idrografici.map&service=WFS"
# TYPENAME = "ID.ACQUEFISICHE.BACINIIDROGRAFICI.PRINCIPALI"

# # 1. Crea connessione WFS (versione 1.1.0 per compatibilità)
# w = WebFeatureService(
#     url=BASE_PCN, 
#     version="1.1.0", 
#     headers=DEFAULT_HEADERS, 
#     timeout=60
# )

# print("[INFO] layers:", len(list(w.contents)))

# # 2. Richiedi features del layer specifico
# resp = w.getfeature(
#     typename=[TYPENAME], 
#     srsname="EPSG:4326"  # Richiedi coordinate geografiche
# )

# # 3. Leggi GML direttamente in GeoDataFrame
# gdf_wfs = gpd.read_file(io.BytesIO(resp.read()))

# gdf_preview(gdf_wfs, name=TYPENAME)
# #safe_save(gdf_wfs, "pcn_bacini_principali")

---

## 3) ArcGIS REST (Esri samples)

### Cos'è


È l'API proprietaria di **Esri** per accedere a dati da ArcGIS Server/Online. Molto diffusa in enti pubblici e aziende.

### Tipi di servizi:
- **MapServer**: Mappe renderizzate (immagini)
- **FeatureServer**: Dati vettoriali interrogabili
- **ImageServer**: Dati raster

### Struttura endpoint:
```
/services                    → Lista servizi
/FeatureServer               → Info servizio
/FeatureServer/{layer}/query → Query dati
```

### Parametri query comuni:
- `where`: Filtro SQL (usa `1=1` per tutto)
- `outFields`: Campi da restituire (`*` = tutti)
- `outSR`: Sistema di coordinate output
- `f`: Formato (json, geojson, pjson)

### Cosa facciamo:
1. Elenchiamo tutti i FeatureServer disponibili
2. Cerchiamo un layer che supporti GeoJSON
3. Scarichiamo i dati del primo layer trovato

In [37]:
# URL base dei servizi demo Esri
# Indici ArcGIS del Trentino = "https://geoservices.provincia.tn.it/agol/rest/services"
BASE_ESRI = "https://services.arcgis.com/V6ZHFr6zdgNZuVG0/ArcGIS/rest/services"

# 1. Leggi indice servizi
idx = fetch_json(BASE_ESRI, params={"f": "json"})

# 2. Filtra solo FeatureServer (servizi vettoriali)
svcs = [s for s in idx.get("services", []) if s.get("type") == "FeatureServer"]
print("[INFO] FeatureServer:", len(svcs))

# Mostra primi 5
svcs[:5]

[INFO] FeatureServer: 2461


[{'name': '100ft buffer of san bernardino  roads by type',
  'type': 'FeatureServer',
  'url': 'https://services.arcgis.com/V6ZHFr6zdgNZuVG0/ArcGIS/rest/services/100ft buffer of san bernardino  roads by type/FeatureServer'},
 {'name': '100ft buffer of san bernardino  roads by type no dissolve',
  'type': 'FeatureServer',
  'url': 'https://services.arcgis.com/V6ZHFr6zdgNZuVG0/ArcGIS/rest/services/100ft buffer of san bernardino  roads by type no dissolve/FeatureServer'},
 {'name': '100ft buffer of San Bernardino Roads',
  'type': 'FeatureServer',
  'url': 'https://services.arcgis.com/V6ZHFr6zdgNZuVG0/ArcGIS/rest/services/100ft buffer of San Bernardino Roads/FeatureServer'},
 {'name': '100ft buffer of san bernardino roads no dissolve no multiscale',
  'type': 'FeatureServer',
  'url': 'https://services.arcgis.com/V6ZHFr6zdgNZuVG0/ArcGIS/rest/services/100ft buffer of san bernardino roads no dissolve no multiscale/FeatureServer'},
 {'name': '100ft buffer of San Bernardino Roads with overlap

### Ricerca layer con supporto GeoJSON

Non tutti i layer ArcGIS supportano GeoJSON (alcuni solo JSON proprietario). Cerchiamo uno che funzioni.

In [38]:
def find_geojson_layer(base, services):
    """Cerca il primo layer che risponde in GeoJSON"""
    for svc in services:
        name = svc["name"]
        
        # Info sul servizio
        info = fetch_json(f"{base}/{name}/FeatureServer", params={"f": "json"})
        
        # Prova ogni layer del servizio
        for lyr in info.get("layers", []):
            q = f"{base}/{name}/FeatureServer/{lyr['id']}/query"
            params = {
                "where": "1=1",      # Nessun filtro
                "outFields": "*",    # Tutti i campi
                "outSR": 4326,       # EPSG:4326
                "f": "geojson"       # Formato richiesto
            }
            
            try:
                gj = fetch_json(q, params=params, timeout=80)
                feats = gj.get("features", [])
                if feats:
                    print("[OK] GeoJSON:", name, lyr["id"], "→", len(feats), "features (parziale)")
                    return q, params
            except Exception as e:
                print("[SKIP]", name, lyr["id"], e)
                
    raise RuntimeError("Nessun layer GeoJSON trovato")

# Cerca e scarica
qurl, qparams = find_geojson_layer(BASE_ESRI, svcs)
gdf_esri = gpd.GeoDataFrame.from_features(
    fetch_json(qurl, params=qparams), 
    crs="EPSG:4326"
)

gdf_preview(gdf_esri, name="esri_sample")
safe_save(gdf_esri, "esri_sample_layer")

[OK] GeoJSON: 100ft buffer of san bernardino  roads by type 0 → 7 features (parziale)
[preview] esri_sample | CRS: EPSG:4326 | righe: 7
Colonne: ['geometry', 'OBJECTID', 'RTTYP', 'Count_', 'AnalysisArea']


,geometry,OBJECTID,RTTYP,Count_,AnalysisArea
0,"MULTIPOLYGON (((-116.97961 35.76414, -116.9775...",1,,23093,1184.781730
1,"MULTIPOLYGON (((-116.04752 35.59308, -116.0456...",2,C,28,6.953090
2,"MULTIPOLYGON (((-115.46575 35.46166, -115.4665...",3,I,19,65.431409
3,"MULTIPOLYGON (((-115.66384 35.80952, -115.6619...",4,M,50345,1806.796747
4,"MULTIPOLYGON (((-117.0156 34.32175, -117.01554...",5,O,124,18.508832


Bounds: [-117.79856935   33.87265707 -114.1332411    35.80951591]
[save]  data/data_lezione_03/esri_sample_layer.gpkg e data/data_lezione_03/esri_sample_layer.parquet


---

## 4) Overpass / OSM – Servizi sanitari a Napoli

### Cos'è OpenStreetMap (OSM)?

È un progetto collaborativo per creare una **mappa mondiale libera**. I dati sono contribuiti da volontari e sono di dominio pubblico (ODbL).

### Cos'è Overpass API?

È un servizio per **interrogare** OSM con query complesse. Non modifica i dati, solo lettura.

### Linguaggio Overpass QL:
```
[out:json][timeout:30];        // Configurazione
(
  node["amenity"="pharmacy"]   // Nodi con tag amenity=pharmacy
  (bbox);                       // In bounding box
);
out center;                     // Output con coordinate centro
```

### Tipi di elementi OSM:
- **node**: Punto singolo (es. farmacia)
- **way**: Linea o area (es. edificio ospedale)
- **relation**: Relazione tra elementi (es. ospedale con più edifici)

### Cosa facciamo:
1. Cerchiamo farmacie e ospedali in un'area di Napoli
2. Creiamo una griglia 1km × 1km
3. Contiamo i POI per cella (analisi densità)
4. Visualizziamo su mappa interattiva

In [9]:
# Server Overpass disponibili (fallback se uno non risponde)
OVERPASS = [
    "https://overpass-api.de/api/interpreter",
    "https://overpass.kumi.systems/api/interpreter"
]

# Query Overpass QL: cerca amenity=pharmacy o hospital nella bbox di Napoli
query = '''
[out:json][timeout:30];
(
  node["amenity"~"pharmacy|hospital"](40.80,14.18,40.92,14.35);
  way["amenity"~"pharmacy|hospital"](40.80,14.18,40.92,14.35);
  relation["amenity"~"pharmacy|hospital"](40.80,14.18,40.92,14.35);
);
out center;
'''

def overpass(q):
    """Esegui query Overpass con fallback su server alternativi"""
    last = None
    for url in OVERPASS:
        try:
            print("[Overpass] →", url)
            r = requests.post(
                url, 
                data={"data": q}, 
                headers=DEFAULT_HEADERS, 
                timeout=60
            )
            r.raise_for_status()
            print("[Overpass] OK")
            return r.json()
        except Exception as e:
            print("[Overpass] fallback:", e)
            last = e
            time.sleep(1.5)
    raise last

# Esegui query
data = overpass(query)

# Estrai coordinate da elementi OSM
rows = []
for el in data.get("elements", []):
    # Nodi hanno lat/lon diretti
    if "lat" in el and "lon" in el:
        lon, lat = el["lon"], el["lat"]
    # Way/relation hanno "center"
    elif "center" in el:
        c = el["center"]
        lon, lat = c["lon"], c["lat"]
    else:
        lon = lat = None
        
    rows.append({
        "id": el.get("id"),
        "amenity": (el.get("tags") or {}).get("amenity"),
        "name": (el.get("tags") or {}).get("name"),
        "lon": lon,
        "lat": lat
    })

# Crea GeoDataFrame
gdf_osm = gpd.GeoDataFrame(
    rows,
    geometry=gpd.points_from_xy(
        pd.Series([r["lon"] for r in rows]),
        pd.Series([r["lat"] for r in rows])
    ),
    crs="EPSG:4326"
).dropna(subset=["geometry"])

gdf_preview(gdf_osm, name="OSM Napoli sanità")
safe_save(gdf_osm, "osm_napoli_sanita")

[Overpass] → https://overpass-api.de/api/interpreter
[Overpass] OK
[preview] OSM Napoli sanità | CRS: EPSG:4326 | righe: 248
Colonne: ['id', 'amenity', 'name', 'lon', 'lat', 'geometry']


,id,amenity,name,lon,lat,geometry
0,439726617,pharmacy,De Benedictis,14.232108,40.844166,POINT (14.23211 40.84417)
1,439726641,pharmacy,None,14.231051,40.843706,POINT (14.23105 40.84371)
2,439726733,pharmacy,Farmacia Cannone,14.232663,40.843851,POINT (14.23266 40.84385)
3,552390833,pharmacy,Farmacia Tufarelli,14.344006,40.843682,POINT (14.34401 40.84368)
4,790306187,pharmacy,Farmacia Mandanici,14.282001,40.903770,POINT (14.282 40.90377)


Bounds: [14.1820495 40.8015112 14.3470023 40.9198166]
[save]  data/data_lezione_03/osm_napoli_sanita.gpkg e data/data_lezione_03/osm_napoli_sanita.parquet


### Analisi di densità su griglia

Creiamo una **griglia regolare** 1km × 1km e contiamo quanti POI cadono in ogni cella. Questo ci mostra le aree con maggiore concentrazione di servizi sanitari.

#### Passaggi:
1. Converti in proiezione metrica (EPSG:3857) per lavorare in metri
2. Crea celle quadrate di 1000m
3. Spatial join tra POI e celle
4. Conta POI per cella
5. Visualizza con opacità proporzionale al conteggio

In [10]:
from math import floor

def make_grid(gdf, km=1.0):
    """Crea griglia regolare in km"""
    # Converti in proiezione metrica Web Mercator
    g = gdf.to_crs(3857)
    minx, miny, maxx, maxy = g.total_bounds
    step = int(km * 1000)  # km → metri
    
    # Genera coordinate angoli celle
    xs = list(range(int(minx), int(maxx)+1, step))
    ys = list(range(int(miny), int(maxy)+1, step))
    
    # Crea poligoni celle
    polys = [box(x, y, x+step, y+step) for x in xs[:-1] for y in ys[:-1]]
    
    return gpd.GeoDataFrame(geometry=polys, crs=3857).to_crs(4326)

# Crea griglia 1km
grid = make_grid(gdf_osm, km=1.0)

# Spatial join: associa ogni POI alla sua cella
join = gpd.sjoin(gdf_osm, grid, how="left", predicate="within")

# Conta POI per cella
agg = join.groupby("index_right").size().rename("count").reset_index()

# Aggiungi conteggi alla griglia
grid["count"] = 0
if len(agg):
    grid.loc[agg["index_right"].dropna().astype(int), "count"] = agg["count"].values

gdf_preview(grid.sort_values("count", ascending=False).head(10), name="Top celle")
safe_save(grid, "grid_napoli_1km_count")

# Mappa interattiva con Folium
m = folium.Map(location=[40.853, 14.305], zoom_start=12)
folium.GeoJson(
    grid.to_json(),
    style_function=lambda f: {
        "weight": 0.2,
        "fillOpacity": min(0.75, (f["properties"].get("count", 0)/10))
    },
    tooltip=folium.GeoJsonTooltip(fields=["count"])
).add_to(m)

m

[preview] Top celle | CRS: EPSG:4326 | righe: 10
Colonne: ['geometry', 'count']


,geometry,count
125,"POLYGON ((14.25391 40.8423, 14.25391 40.84909,...",12
91,"POLYGON ((14.23594 40.8423, 14.23594 40.84909,...",11
160,"POLYGON ((14.27188 40.84909, 14.27188 40.85589...",10
126,"POLYGON ((14.25391 40.84909, 14.25391 40.85589...",10
124,"POLYGON ((14.25391 40.8355, 14.25391 40.8423, ...",8


Bounds: [14.21797736 40.82870209 14.27187628 40.85588556]
[save]  data/data_lezione_03/grid_napoli_1km_count.gpkg e data/data_lezione_03/grid_napoli_1km_count.parquet


---

## 5) GeoJSON via HTTP – Natural Earth Populated Places (Italy)

### Cos'è GeoJSON?

È un **formato JSON standardizzato** (RFC 7946) per rappresentare geometrie geografiche. Molto popolare per web mapping.

### Struttura GeoJSON:
```json
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [12.4964, 41.9028]
      },
      "properties": {
        "name": "Roma",
        "population": 2872800
      }
    }
  ]
}
```

### Natural Earth

Natural Earth è un **dataset cartografico pubblico** di alta qualità. Include:
- Confini amministrativi
- Città e luoghi popolati
- Fiumi, laghi, oceani
- Elementi culturali

Disponibile su GitHub in vari formati, incluso GeoJSON.

### Cosa facciamo:
1. Scarichiamo GeoJSON dei luoghi popolati mondiali
2. Filtriamo solo l'Italia
3. Salviamo il subset

In [11]:
# URL GeoJSON Natural Earth su GitHub
NE_PP = "https://raw.githubusercontent.com/nvkelso/natural-earth-vector/master/geojson/ne_50m_populated_places_simple.geojson"

# 1. Scarica GeoJSON completo (mondiale)
pp = fetch_json(NE_PP)

# 2. Converti in GeoDataFrame
gdf_pp = gpd.GeoDataFrame.from_features(pp, crs="EPSG:4326")

# 3. Filtra solo Italia usando la colonna adm0name (nome paese)
ita = gdf_pp[gdf_pp["adm0name"] == "Italy"].copy()

gdf_preview(ita, name="NE populated places (Italy)")
safe_save(ita, "ne_populated_places_italy")

[fetch_json] 1/2 → https://raw.githubusercontent.com/nvkelso/natural-earth-vector/master/geojson/ne_50m_populated_places_simple.geojson


[fetch_json] OK
[preview] NE populated places (Italy) | CRS: EPSG:4326 | righe: 21
Colonne: ['geometry', 'scalerank', 'natscale', 'labelrank', 'featurecla', 'name', 'namepar', 'namealt', 'nameascii', 'adm0cap', 'capalt', 'capin', 'worldcity', 'megacity', 'sov0name', 'sov_a3', 'adm0name', 'adm0_a3', 'adm1name', 'iso_a2', 'note', 'latitude', 'longitude', 'pop_max', 'pop_min', 'pop_other', 'rank_max', 'rank_min', 'meganame', 'ls_name', 'min_zoom', 'ne_id']


,geometry,scalerank,natscale,labelrank,featurecla,name,namepar,namealt,nameascii,adm0cap,...,longitude,pop_max,pop_min,pop_other,rank_max,rank_min,meganame,ls_name,min_zoom,ne_id
2,POINT (15.799 40.642),10,1,3,Admin-1 region capital,Potenza,None,None,Potenza,0,...,15.798997,69060,69060,0.0,8,8,None,None,7.0,1159117259
3,POINT (14.656 41.563),10,1,3,Admin-1 region capital,Campobasso,None,None,Campobasso,0,...,14.655997,50762,50762,0.0,8,8,None,None,7.0,1159117283
4,POINT (7.315 45.737),10,1,3,Admin-1 region capital,Aosta,None,None,Aosta,0,...,7.315003,34062,34062,0.0,7,7,None,None,7.0,1159117361
20,POINT (11.34002 44.50042),7,20,3,Admin-1 region capital,Bologna,None,None,Bologna,0,...,11.340021,488172,371217,467366.0,10,10,None,Bologna,6.1,1159139461
21,POINT (9.10398 39.2224),7,20,3,Admin-1 region capital,Cagliari,None,None,Cagliari,0,...,9.103982,291511,164249,282477.0,10,9,None,Cagliari,6.1,1159139469


Bounds: [ 7.315003 37.499971 16.872758 46.080429]
[save]  data/data_lezione_03/ne_populated_places_italy.gpkg e data/data_lezione_03/ne_populated_places_italy.parquet


---

# Use case: Parcheggi vs domanda potenziale — densità di civici entro 250 m


> Obiettivo: stimare la **domanda potenziale** attorno ai parcheggi pubblici come conteggio di **numeri civici** entro **250 m**.  
> Fonti: **Comune di Trento** (endpoint REST GeoJSON ufficiali; sostituibili con WFS/ArcGIS se disponibili).  
> Output: KPI per parcheggio (`civici_250m`, opz. `civici_per_posto`) + mappa Folium + export GeoJSON/CSV.

**Nota esecuzione**: le celle qui sotto scaricano dati da internet. Se esegui offline, definisci `DATA_DIR` e carica localmente gli stessi layer in formato GeoJSON.


> Riferimenti dati:
> - Parcheggi auto (Comune di Trento, endpoint GeoJSON): https://gis.comune.trento.it/dbexport?db=base&sc=mobilita&ly=parcheggi_auto&fr=geojson
> - Numeri civici (Comune di Trento, endpoint GeoJSON): https://gis.comune.trento.it/dbexport?db=base&sc=toponomastica&ly=civici_web&fr=geojson
> - Portale cartografico e guida a WMS/WFS del Comune: https://www.comune.trento.it/aree-tematiche/cartografia


In [2]:
# === Config & import ===
import os
import math
import json
import warnings
from pathlib import Path

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
import folium

warnings.filterwarnings("ignore")

# Endpoints ufficiali Comune di Trento (GeoJSON REST).
CIVICI_URL = "https://gis.comune.trento.it/dbexport?db=base&sc=toponomastica&ly=civici_web&fr=geojson"
PARCHEGGI_URL = "https://gis.comune.trento.it/dbexport?db=base&sc=mobilita&ly=parcheggi_auto&fr=geojson"

# Parametri
BUFFER_METERS = 250            # raggio (modificabile)
CRS_METRIC = 32632             # UTM 32N 
CRS_LATLON = 4326

# Cartella export 
DATA_DIR = Path("data/data_lezione_03/data_parcheggi_usecase")
DATA_DIR.mkdir(exist_ok=True)

print("Config pronta.")

Config pronta.


In [3]:
# Opzione: via web/ fallback su daticaricati
from pathlib import Path
from zipfile import ZipFile
import geopandas as gpd

def read_geojson_from_zip(zip_path: Path):
    """Apre uno ZIP e legge il primo GeoJSON trovato (preferenza *.geojson, poi *.json)."""
    zip_path = Path(zip_path)
    if not zip_path.exists():
        raise FileNotFoundError(f"ZIP non trovato: {zip_path}")

    # Elenco dei file interni allo zip
    with ZipFile(zip_path) as zf:
        names = zf.namelist()

    # Preferisci *.geojson, altrimenti accetta *.json
    candidates = [n for n in names if n.lower().endswith(".geojson")]
    if not candidates:
        candidates = [n for n in names if n.lower().endswith(".json")]

    if not candidates:
        raise ValueError(f"Nessun file GeoJSON trovato in: {zip_path}")

    target = candidates[0]
    # Usa il virtual filesystem di Fiona/GDAL: vfs="zip://<path-zip>"
    return gpd.read_file(target, vfs=f"zip://{zip_path}")

# -------- uso --------
try:
    gdf_civici = gpd.read_file(CIVICI_URL)
    gdf_parcheggi = gpd.read_file(PARCHEGGI_URL)
    print("Dati letti dalle URL.")
except Exception as e:
    print("⚠️ Download non riuscito.", e)
    print("Provo a leggere da ZIP locali in", DATA_DIR)
    gdf_civici = read_geojson_from_zip(DATA_DIR / "civici_web.zip")
    gdf_parcheggi = read_geojson_from_zip(DATA_DIR / "parcheggi_auto.zip")
    print("Dati letti da ZIP locali.")

print("civici:", len(gdf_civici), "| parcheggi:", len(gdf_parcheggi))


Dati letti dalle URL.
civici: 40152 | parcheggi: 22


In [14]:
gdf_civici.head(5)

,civico_num,civico_let,civico_alf,desvia,strada,cap,tipo_num,tipo_en,ingresso,ingr_en,fumetto,url,sobborgo,geometry
0,8,,8,VIA AOSTA,160,38122,principale,front entrance,altro,other,"VIA AOSTA, 8",codiceVia=160&numeroCivico=8&barraCivico=,Trento,POINT (664594.124 5102545.972)
1,1,,1,VIA BORINO,7335,38123,secondario,side entrance,altro,other,"VIA BORINO, 1",codiceVia=7335&numeroCivico=1&barraCivico=,Povo,POINT (667251.083 5104155.428)
2,20,,20,VIA CELVA,7455,38123,secondario,side entrance,altro,other,"VIA CELVA, 20",codiceVia=7455&numeroCivico=20&barraCivico=,Povo,POINT (668250.43 5104373.69)
3,1,,1,VIA CAURIOL,660,38122,secondario,side entrance,cancello,gate,"VIA CAURIOL, 1",codiceVia=660&numeroCivico=1&barraCivico=,Trento,POINT (664204.701 5102297.76)
4,1,,1,VIA E. CHINI,770,38123,secondario,side entrance,cancello,gate,"VIA E. CHINI, 1",codiceVia=770&numeroCivico=1&barraCivico=,Trento,POINT (664785.025 5102168.036)


In [15]:
gdf_parcheggi.head(3)

,id,nome,indirizzo,tipologia,regolament,categoria,gestione,posti,link,p_rotazione,...,p_carico-scarico,p_elettriche,orario_lunven,orario_sabato,orario_festivo,link_gmaps,regol_xweb,taxi,riservati altre categorie,geometry
0,1,Garage Autorimessa Europa,"VIA DELLA ROGGIA GRANDE, 20",coperto,pagamento,non attestamento,terzi,70,https://www.comune.trento.it/Vivere-il-comune/...,60.0,...,None,NaN,n.d.,n.d.,n.d.,None,None,None,None,POINT (664359.522 5103780.8)
1,16,Garage Autosilo Buonconsiglio - P3,"VIA F. PETRARCA, 1/5",coperto,pagamento,non attestamento,terzi,494,https://www.comune.trento.it/Vivere-il-comune/...,188.0,...,None,NaN,00:00-24:00,00:00-24:00,00:00-24:00,https://maps.app.goo.gl/qVHmRxz8VeksTDrp7,a pagamento,None,None,POINT (664279.151 5104454.924)
2,11,Garage Parcheggio Duomo - P5,"PIAZZA E. MOSNA, 1/A",coperto,pagamento,non attestamento,terzi,198,https://www.comune.trento.it/Vivere-il-comune/...,119.0,...,None,NaN,00:00-24:00,00:00-24:00,00:00-24:00,https://www.google.com/maps/place/Parcheggio+D...,a pagamento,None,None,POINT (663600.779 5103699.785)


In [16]:
# Pulizia minima
gdf_civici = gdf_civici[gdf_civici.geometry.notnull()].copy()
gdf_parcheggi = gdf_parcheggi[gdf_parcheggi.geometry.notnull()].copy()

# Reproject in metrico
gdf_civici = gdf_civici.to_crs(epsg=CRS_METRIC)
gdf_parcheggi = gdf_parcheggi.to_crs(epsg=CRS_METRIC)


# estraiamo la colonna con la capacità dei parcheggi
capacity_col = "posti"


gdf_parcheggi["capacity_col"] = (
    pd.to_numeric(gdf_parcheggi[capacity_col], errors="coerce")
      .fillna(0)
      .astype(int)
)


In [17]:
# Buffer individuale per ciascun parcheggio
buf = gdf_parcheggi.copy()
buf["geometry"] = buf.geometry.buffer(BUFFER_METERS)
buf = buf.set_geometry("geometry")

# Spatial join: civici entro buffer
join = gpd.sjoin(gdf_civici, buf[["geometry"]], how="left", predicate="within")

# Conteggio civici per parcheggio (index_right è l'indice del buffer/parcheggio)
counts = join.groupby("index_right").size().rename("civici_250m")

# Unisci ai parcheggi originali
res = gdf_parcheggi.copy()
res["civici_250m"] = counts.reindex(res.index).fillna(0).astype(int)

# KPI opzionale: civici per posto auto (se capacity disponibile e >0)
if capacity_col is not None and capacity_col in res.columns:
    cap = pd.to_numeric(res[capacity_col], errors="coerce").fillna(0)
    res["civici_per_posto"] = (res["civici_250m"] / cap.replace(0, pd.NA)).round(2)
else:
    res["civici_per_posto"] = pd.NA

# Top 10 per domanda
top10 = res.sort_values("civici_250m", ascending=False).head(10)[["civici_250m","civici_per_posto"] + ([capacity_col] if capacity_col else [])]
top10.reset_index(drop=True, inplace=True)
top10

,civici_250m,civici_per_posto,posti
0,1136,16.23,70
1,783,1.70,460
2,634,1.28,494
3,338,0.77,440
4,282,1.25,225
5,272,1.74,156
6,263,1.33,198
7,218,1.33,164
8,202,0.61,329
9,192,1.12,171


In [18]:
#  Export risultati 
res_latlon = res.to_crs(epsg=CRS_LATLON)
out_geojson = DATA_DIR / "parcheggi_domanda_250m.geojson"
out_csv = DATA_DIR / "parcheggi_domanda_250m.csv"

res_latlon.to_file(out_geojson, driver="GeoJSON")
res_latlon.drop(columns="geometry").to_csv(out_csv, index=False, encoding="utf-8")

print("Salvato:", out_geojson, "e", out_csv)

Salvato: data_parcheggi_usecase/parcheggi_domanda_250m.geojson e data_parcheggi_usecase/parcheggi_domanda_250m.csv


In [19]:
# Mappa Folium

center_geom = res.to_crs(CRS_LATLON).geometry.union_all()
center = [center_geom.centroid.y, center_geom.centroid.x]  # [lat, lon]

m = folium.Map(location=center, zoom_start=13, control_scale=True)

# Prepariamo una scala semplice su 4 quantili
q = res["civici_250m"].quantile([0, 0.25, 0.5, 0.75, 1]).tolist()

def color_for(val):
    if math.isnan(val):
        return "#cccccc"
    if val <= q[1]: return "#edf8fb"
    if val <= q[2]: return "#b2e2e2"
    if val <= q[3]: return "#66c2a4"
    return "#238b45"

def style_fn(feature):
    val = feature["properties"].get("civici_250m", None)
    return {
        "fillColor": color_for(val),
        "color": "#555555",
        "weight": 1,
        "fillOpacity": 0.6,
    }

# GeoJson dei parcheggi (geometria originale) con popup
gj = folium.GeoJson(
    res.to_crs(epsg=CRS_LATLON),
    name="Parcheggi",
    style_function=style_fn,
    tooltip=folium.GeoJsonTooltip(fields=["civici_250m"] + ([capacity_col] if capacity_col else []),
                                  aliases=["Civici entro 250m"] + (["Posti"] if capacity_col else []),
                                  sticky=True)
).add_to(m)

folium.LayerControl().add_to(m)

m